To automatically predict the congestion level on-board of public transport via open data eg. Trafiklab/Google and data pipeline powered by IBM and other open source community 

Step 1. Read the ID from station names

In [25]:
from collections import OrderedDict
import csv
import pandas as pd
import numpy as np
import requests 

import xml.etree.ElementTree as ET
from lxml.etree import fromstring

In [ ]:
dt = pd.read_excel('list_stop_sl_30.xlsx')
#dt = pd.DataFrame(dt)
r = [0 for i in range(len(dt.index))]



for index, row in dt.iterrows():
    urls = "https://api.resrobot.se/v2/location.name?key=XXX&input=" + row['stopName']
    r[index] = requests.get(urls)

print(r[0].status_code)    


In [ ]:
print(r[:2])
#status0 = np.array(r.status_code)
#print(np.unique(status0))


In [ ]:
dfcols = ['name','ID','lon','lat']
df = pd.DataFrame(columns = dfcols)
    
for el in r:
    root = ET.fromstring(el.content)

#for child in root.iter('*'):
#    print(child.attrib,child.attrib)

    for child in root.iter('{hafas_rest_v1}StopLocation'):
        name = child.attrib['name']
        ID = child.attrib['id']
        lon = child.attrib['lon']
        lat = child.attrib['lat']
        df = df.append(pd.Series([name, ID, lon,lat], index = dfcols), ignore_index = True)

print(df.shape)
df.head()

In [ ]:
import pickle

with open('dataset_30_station_ID.pickle', 'wb') as output:
    pickle.dump(df, output)

df.to_excel('dataset_30_station_ID.xlsx')

In [ ]:
import pickle

# Load data (same as pickled output)
with open('dataset_30_station_ID.pickle', 'rb') as data:
    df = pickle.load(data)

df.head()

In [ ]:
##SJ: working version 1
#url1 = "https://api.resrobot.se/v2/location.name?"

#params1 = OrderedDict([('key','XXX'),('input','Centralen')])
#response1 = requests.get(url1, params=params1)

#print(response1.status_code)

#import pandas as pd
#import xml.etree.ElementTree as ET
#from lxml.etree import fromstring

#root = ET.fromstring(response.content)
#for child in root.iter('{hafas_rest_v1}StopLocation'):
#    print(child.attrib['id'],child.attrib['name'])

#dfcols = ['name','ID','lon','lat']
#df = pd.DataFrame(columns = dfcols)

#for child in root.iter('{hafas_rest_v1}StopLocation'):
#    name = child.attrib['name']
#    ID = child.attrib['id']
#    lon = child.attrib['lon']
#    lat = child.attrib['lat']
#    df = df.append(pd.Series([name, ID, lon,lat], index = dfcols), ignore_index = True)

#df.head()
print(df.shape[0])

step 2 To get public transport (realtime info) for arrival and departure time at the searched ID

In [ ]:
r2 = [0 for i in range(df.shape[0])]

for index, row in df.iterrows():
    urls2 = "https://api.resrobot.se/v2/departureBoard?key=XXX&id=" + row['ID']
    r2[index] = requests.get(urls2)

print(r2[0].status_code)  

#url2 = "https://api.resrobot.se/v2/departureBoard?key=XXX&id=740020749"
#response2 = requests.get(url2)


In [ ]:
dfcols2 = ['name','stop','stopid','direction','transportNumber','date','time']
df2 = pd.DataFrame(columns = dfcols2)

for el in r2:
    root2 = ET.fromstring(el.content)

    for child in root2.iter('{hafas_rest_v1}Departure'):
        lineName = child.attrib['name']
        stopName = child.attrib['stop']
        stopID = child.attrib['stopid']
        lineDirection = child.attrib['direction']
        lineNumber = child.attrib['transportNumber']
        date = child.attrib['date']
        time = child.attrib['time']    
        df2 = df2.append(pd.Series([lineName,stopName,stopID,lineDirection,lineNumber,date,time], index = dfcols2), ignore_index = True)

df2.head()

#root2 = ET.fromstring(response2.content)
#for child in root2.iter('{hafas_rest_v1}Departure'):
#    print(child.tag,child.attrib)

In [ ]:
import pickle

with open('dataset_30_ID_departure_time.pickle', 'wb') as output:
    pickle.dump(df2, output)

df2.to_excel('dataset_30_ID_departure_time.xlsx')

In [ ]:
# Load data (same as pickled output)
with open('dataset_30_ID_departure_time.pickle', 'rb') as data:
    df2 = pickle.load(data)

df2.head()

In [2]:
url3 = "https://api.sl.se/api2/deviations.xml?" #json instead of xml

#SJ: interested in bus that usually been over-loaded
params3 = OrderedDict([('key','XXX'),('transportMode','bus'),('siteId','9192')]) #
response3 = requests.get(url3, params=params3)

print(response3.status_code)
print(response3.content)

200
b'<?xml version="1.0" encoding="utf-8"?><ResponseOfListOfDeviation xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance"><StatusCode>0</StatusCode><ExecutionTime>51</ExecutionTime><ResponseData><Deviation><Created>2020-01-16T15:08:42.107+01:00</Created><MainNews>false</MainNews><SortOrder>1</SortOrder><Header>Flyttade h\xc3\xa5llplatser f\xc3\xb6r linje 163 och 180</Header><Details>F\xc3\xb6r\xc3\xa4ndringar f\xc3\xb6r linje 163 och 180 i K\xc3\xa4rrtorp fr\xc3\xa5n 21 januari p\xc3\xa5 grund av ombyggnation av busstorget. Detta ber\xc3\xa4knas p\xc3\xa5g\xc3\xa5 till och med maj 2021.\r\n\r\nH\xc3\xa5llplatsen f\xc3\xb6r linje 163 flyttas upp till L\xc3\xa5gsk\xc3\xa4rsv\xc3\xa4gen utanf\xc3\xb6r Coop.\r\nH\xc3\xa5llplatsen f\xc3\xb6r linje 180 flyttas upp till L\xc3\xa5gsk\xc3\xa4rsv\xc3\xa4gen utanf\xc3\xb6r Kulturskolan.</Details><Scope>Ombyggnation i K\xc3\xa4rrtorp</Scope><DevCaseGid>9076001019795870</DevCaseGid><DevMessageVersionN

In [26]:
root3 = ET.fromstring(response3.content)
print(root3.tag)

N = 0
for index, child in enumerate(root3.iter('Scope')):
    print(index,child.text)
    N = N + 1

dfcols3 = ['ScopeElements','Scope','FromDateTime','UpToDateTime','Created','Details']
df3 = pd.DataFrame(index=range(N),columns = dfcols3)
df3 = df3.fillna(0)

print(dfcols3[0])

##for el in r2:
##    root2 = ET.fromstring(el.content)
for col in range(len(dfcols3)):
    for index, child in enumerate(root3.iter(dfcols3[col])):
        df3.iloc[[index],[col]] = child.text 

import pickle

with open('dataset_traffic_infov1.pickle', 'wb') as output:
    pickle.dump(df3, output)

from openpyxl.workbook import Workbook

df3.to_excel('dataset_traffic_infov1.xlsx')

df3.head()

ResponseOfListOfDeviation
0 Ombyggnation i Kärrtorp
1 Blåbuss 2; Buss 55, 76, 96, 492, 496, 497
2 Buss 491
ScopeElements


,ScopeElements,Scope,FromDateTime,UpToDateTime,Created,Details
0,"Buss 163, 180, 194",Ombyggnation i Kärrtorp,2020-01-21T05:00:00,2021-05-30T23:59:00,2020-01-16T15:08:42.107+01:00,Förändringar för linje 163 och 180 i Kärrtorp ...
1,"Blåbuss 2; Buss 55, 76, 96, 492, 496, 497","Blåbuss 2; Buss 55, 76, 96, 492, 496, 497",2020-01-08T17:31:35.853,2020-08-24T16:00:00,2020-01-08T17:31:35.883+01:00,Hållplats Slussen är flyttad ca 75 meter framå...
2,Buss 491,Buss 491,2019-12-13T00:00:00,2020-08-24T23:30:00,2019-12-10T20:19:17.9+01:00,Hållplats Slussen är flyttad ca 75 meter för b...


In [50]:
df3.head()

,ScopeElements,Scope,FromDateTime,UpToDateTime,Created,Details


In [ ]:

    stopName = child.attrib['stop']
    stopID = child.attrib['stopid']
    lineDirection = child.attrib['direction']
    lineNumber = child.attrib['transportNumber']
    date = child.attrib['date']
    time = child.attrib['time']    
    df2 = df2.append(pd.Series([lineName,stopName,stopID,lineDirection,lineNumber,date,time], index = dfcols2), ignore_index = True)

df2.head()

In [ ]:
root_tmp = ET.fromstring(response2.content)
for child in root_tmp.iter('{hafas_rest_v1}Stops'):
    for gchild in child.iter('*'):
        print(gchild.tag,gchild.attrib)